# Tutorial on Data Wrangling

In [1]:
import pandas as pd
import numpy as np

In [2]:
# connect to Snowflake
from snowflake.snowpark import Session
import snowflake.connector
from snowflake_connector import account, user, password, warehouse, database, schema, region

# connection
ctx = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    region=region,
    role = 'CPDA_ROLE',
    warehouse='CPDA_WH',
    passcode='943885'
)

# cursor
cs = ctx.cursor() # cursor

## Concat String to Array
Step 1: read data into pandas dataframe

In [37]:
sql_query = """
SELECT * 
FROM PDI_DATA_PRODUCTS.PERSON_DATA_PROFILE.PERSON_BMC_STATUS
LIMIT 1000;
"""

# execute and store query
cs.execute(sql_query)
df = cs.fetch_pandas_all()

df.columns = [x.lower() for x in df.columns]
df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,ap,biology,chemistry,college_algebra,economics,physics,active_subscriber
0,ffffffff518f9019e4b0791f62fa9242,False,True,False,False,False,False,False,False,False,False
1,ffffffff612559b148fcfa2c37a45906,False,True,False,False,False,False,False,False,False,False
2,ffffffff61db71175c28994883adf968,False,True,False,False,False,False,False,False,False,False
3,ffffffff611d2f1b8ed8785a8234763a,False,True,False,False,False,False,False,False,False,False
4,e565e66969644244ba8c981d5095e065,False,True,False,False,False,False,False,False,False,False


In [38]:
# check column data types
df.dtypes

person_id            object
pplus_subscriber       bool
mlm_user               bool
hybrid_user            bool
ap                     bool
biology                bool
chemistry              bool
college_algebra        bool
economics              bool
physics                bool
active_subscriber      bool
dtype: object

Step 2: replicate query:

as ( <br>
    $\;\;\;\;\;\;$ select <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ PERSON_ID, <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ PPLUS_SUBSCRIBER, <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ MLM_USER, <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ HYBRID_USER, <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ ACTIVE_SUBSCRIBER, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when AP='FALSE' THEN '' else 'AP' end as is_ap, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when BIOLOGY='FALSE' THEN '' else 'BIOLOGY' end as is_biology, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when CHEMISTRY='FALSE' THEN '' else 'CHEMISTRY' end as is_chemistry, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when COLLEGE_ALGEBRA='FALSE' THEN '' else 'COLLEGE_ALGEBRA' end as is_college_algebra, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when ECONOMICS='FALSE' THEN '' else 'ECONOMICS' end as is_economics, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ case when PHYSICS='FALSE' THEN '' else 'PHYSICS' end as is_physics <br>
$\;\;\;\;\;\;$ from PDI_DATA_PRODUCTS.PERSON_DATA_PROFILE.PERSON_BMC_STATUS


In [ ]:
# numpy.where(condition, [x, y, ]/)
# add is_ap column
df['is_ap'] = np.where(
    ~df.ap, 
    '',
    'AP'
)

# add is_biology
df['is_biology'] = np.where(
    ~df.biology, 
    '',
    'Biology'
)

# or maybe there's an easier way . . .
df.head()

There's a faster way!

In [52]:
# the for loop!
cols = ['biology', 'ap', 'chemistry', 'college_algebra', 'economics', 'physics']
for col in cols:
    df['is_' + col] = np.where(
        ~df[col], 
        '',
        col
    )

# keep columns not in cols
df = df[
    [col for col in df.columns if col not in cols]
]

df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,active_subscriber,is_ap,is_biology,is_chemistry,is_college_algebra,is_economics,is_physics
0,ffffffff518f9019e4b0791f62fa9242,False,True,False,False,,,,,,
1,ffffffff612559b148fcfa2c37a45906,False,True,False,False,,,,,,
2,ffffffff61db71175c28994883adf968,False,True,False,False,,,,,,
3,ffffffff611d2f1b8ed8785a8234763a,False,True,False,False,,,,,,
4,e565e66969644244ba8c981d5095e065,False,True,False,False,,,,,,


## Index Match

select <br>
$\;\;\;\;\;\;$ PPLUS_SUBSCRIBER, <br>
$\;\;\;\;\;\;$ MLM_USER, <br>
$\;\;\;\;\;\;$ HYBRID_USER, <br>
$\;\;\;\;\;\;$ ACTIVE_SUBSCRIBER, <br>
$\;\;\;\;\;\;$ case when PPLUS_SUBSCRIBER='FALSE' and MLM_USER='TRUE' and  HYBRID_USER='FALSE' THEN 'MLM' <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ when PPLUS_SUBSCRIBER='TRUE' and MLM_USER='FALSE' and  HYBRID_USER='FALSE' THEN 'P+' <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ when PPLUS_SUBSCRIBER='FALSE' and MLM_USER='FALSE' and  HYBRID_USER='FALSE' THEN 'Non' <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ when PPLUS_SUBSCRIBER='TRUE' and MLM_USER='TRUE' and  HYBRID_USER='TRUE' THEN 'Hybrid' <br>
        $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ end as business_model_code, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_ap, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_biology, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_chemistry, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_college_algebra, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_economics, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ is_physics, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ concat_ws(' ',is_ap, is_biology, is_chemistry, is_college_algebra, is_economics, is_physics) as concat_categories, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ strtok_to_array(concat_ws(' ',is_ap, is_biology, is_chemistry, is_college_algebra, is_economics, is_physics)) as string_to_array, <br>
    $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ array_size(strtok_to_array(concat_ws(' ',is_ap, is_biology, is_chemistry, is_college_algebra, is_economics, is_physics))) as different_categories <br>
        $\;\;\;\;\;\;$ from PROD_ANALYTICS_COE_ETEXT.SANDBOX.SPERRY_LIST_CONCAT_OPT <br>

In [53]:
# start over for this step
sql_query = """
SELECT * 
FROM PDI_DATA_PRODUCTS.PERSON_DATA_PROFILE.PERSON_BMC_STATUS
LIMIT 1000;
"""

# execute and store query
cs.execute(sql_query)
df = cs.fetch_pandas_all()

df.columns = [x.lower() for x in df.columns]
df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,ap,biology,chemistry,college_algebra,economics,physics,active_subscriber
0,ffffffff5f5f0281dc0e8201da49e9bf,False,True,False,False,False,False,True,False,True,False
1,ffffffff63322fa26814d77189d4e2af,False,True,False,False,False,False,True,False,True,False
2,ffffffff60414ad0addd0c3e519d8e9a,False,True,False,False,False,False,True,False,True,False
3,ffffffff60d7f33988a7b329b45cd4d6,False,True,False,False,False,False,True,False,True,False
4,ffffffff632d547a2cd9e942baddfe08,False,True,False,False,False,False,True,False,True,False


Add column of arrays with two lines of code:

In [68]:
df[cols].apply(lambda x: x['biology'], axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
995     True
996     True
997     True
998     True
999     True
Length: 1000, dtype: bool

In [74]:
# again, we need our columns
cols = np.array(['biology', 'ap', 'chemistry', 'college_algebra', 'economics', 'physics'])
# cols = ['biology', 'ap', 'chemistry', 'college_algebra', 'economics', 'physics']

# add column of arrays
df['string_to_array'] = df[cols].apply(lambda x: cols[x], axis=1)
df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,ap,biology,chemistry,college_algebra,economics,physics,active_subscriber,string_to_array
0,ffffffff5f5f0281dc0e8201da49e9bf,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]"
1,ffffffff63322fa26814d77189d4e2af,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]"
2,ffffffff60414ad0addd0c3e519d8e9a,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]"
3,ffffffff60d7f33988a7b329b45cd4d6,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]"
4,ffffffff632d547a2cd9e942baddfe08,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]"


Count the length of each array, add as new column:

In [90]:
a = []
for val in df.string_to_array:
    a.append(len(val))

df['new_col'] = a
df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,ap,biology,chemistry,college_algebra,economics,physics,active_subscriber,string_to_array,new_col
0,ffffffff5f5f0281dc0e8201da49e9bf,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2
1,ffffffff63322fa26814d77189d4e2af,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2
2,ffffffff60414ad0addd0c3e519d8e9a,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2
3,ffffffff60d7f33988a7b329b45cd4d6,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2
4,ffffffff632d547a2cd9e942baddfe08,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2


In [101]:
df['n_categories'] = df['string_to_array'].apply(lambda x: len(x))
df.head()

,person_id,pplus_subscriber,mlm_user,hybrid_user,ap,biology,chemistry,college_algebra,economics,physics,active_subscriber,string_to_array,new_col,n_categories
0,ffffffff5f5f0281dc0e8201da49e9bf,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2,2
1,ffffffff63322fa26814d77189d4e2af,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2,2
2,ffffffff60414ad0addd0c3e519d8e9a,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2,2
3,ffffffff60d7f33988a7b329b45cd4d6,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2,2
4,ffffffff632d547a2cd9e942baddfe08,False,True,False,False,False,False,True,False,True,False,"[college_algebra, physics]",2,2


Group by sting_to_array, n_categories and count unique users

In [ ]:
# does not work
# df.groupby(['string_to_array', 'n_categories'])['person_id'].nunique()

df['string_to_array'] = df.string_to_array.astype(str)
df = df\
    .groupby(['string_to_array', 'n_categories'], as_index=False)['person_id']\
        .nunique()\
            .sort_values('n_categories')

df.head(20)

In [ ]:
# create identifier
df['array_id'] = df.string_to_array.astype(str).rank(method='dense').astype(int)
df.head()

In [ ]:
# group by operation
df = df\
    .groupby(['array_id', 'n_categories'], as_index=False)['person_id']\
        .nunique()\
            .merge( # merge with crosswalk
                df.loc[~df.array_id.duplicated(), ['array_id', 'string_to_array']], 
                left_on='array_id', 
                right_on='array_id', 
                how='left')\
                .sort_values('n_categories')

# select columns
df.drop(columns='array_id', inplace=True)
df.head()

## Tab 2: Group by with Dist

In [3]:
df = pd.read_csv('result.csv')
df.columns = [x.lower() for x in df.columns]

df.head()

,ga_client_id,person_id
0,1406024597,ffffffff6318e66337d5b229fc52cbf1
1,80139976.17,ffffffff6318e66337d5b229fc52cbf1
2,10225384.17,ffffffff6130cf2076ed8c5874971d89
3,1022344.163,anonymous
4,1106091.164,anonymous


- split_part(ga_client_id,'.',1) as new_ga_client_id

- SUM(CASE WHEN person_id like '%Unauthenticated%' or person_id like '%anonymous%' THEN 0 ELSE 1 END) over (PARTITION BY split_part(ga_client_id,'.',1)) as ga_id_signed_in_counter

- group by 1,2,3

- order by 4 desc,2 

#### Part 1

In [31]:
search_words = ["Unauthenticated", "anonymous"]
x = df.person_id[0]
np.array([x in search_word for search_word in search_words]).any()

a = pd.Series(['a', 'b', 'b'])
search_words ['a', 'b']
a.str.contains('a')

0     True
1    False
2    False
dtype: bool

In [35]:
','.join(search_words).replace(',', '|')

'Unauthenticated|anonymous'

In [76]:
# split string and get first element in list
df['new_ga_client_id'] = df.ga_client_id.apply(lambda x: x.split('.')[0])

# group by columns
group = ['new_ga_client_id']

search_words = ["Unauthenticated", "anonymous"]
#df['transform'] = 

df['ga_id_signed_in_counter'] = df.groupby(group).person_id\
    .transform(lambda x: (~x.str.contains('Unauthenticated|anonymous', regex=True)).sum())

# df.new_ga_client_id.value_counts()

# df.groupby(group).person_id\
#     .transform(lambda x: x.count())

df.sort_values('person_id', ascending=False)

,ga_client_id,person_id,new_ga_client_id,transform,ga_id_signed_in_counter
45,110052.1662,ffffffff63447c8805c03c0e9c8e14f9,110052,1,2
95,1093099.167,ffffffff633d25b4e699bb280ab9fd2d,1093099,1,1
143,2041817578,ffffffff632de49ae8a29a0ea95e2f59,2041817578,1,1
26,1120324.166,ffffffff632de49ae8a29a0ea95e2f59,1120324,1,1
34,273459866.2,ffffffff632de49ae8a29a0ea95e2f59,273459866,1,1
...,...,...,...,...,...
121,10156365.17,3dc59afa576e4c2a9ace48c3811ccd3e,10156365,1,1
184,1090768.166,3c6eb5f9c6934ee390f124d89148a636,1090768,1,1
8,1113065.167,374e79ff148e4f67a4473f7c8c848a8f,1113065,1,1
160,1142964.166,3385f8bc4cfe486fa8caac595f3206fd,1142964,1,1


In [ ]:
# split string and get first element in list
df['new_ga_client_id'] = df.ga_client_id.apply(lambda x: x.split('.')[0])

# group by columns
group = ['person_id', 'ga_client_id', 'new_ga_client_id']
df['ga_id_signed_in_counter'] = df.groupby(group).person_id.transform(lambda x: x.str.contains('Unauthenticated').sum())

df.sort_values('ga_id_signed_in_counter', ascending=False).head()

#### Part 2

Apply transforms to derived table:

- CASE WHEN person_id like '%Unauthenticated%' or person_id like '%anonymous%' THEN 'signed_out' ELSE 'signed_in' end as is_signed_in, 

- case when person_id like '%anonymous%' THEN '' else listagg(new_ga_client_id, ', ') within GROUP (ORDER BY new_ga_client_id) over(PARTITION BY person_id) end as list_ga_id_per_person

- where ga_id_signed_in_counter=1

- order by 3 desc,2

In [ ]:
# df['is_signed_in'] = 
df.person_id.apply(lambda x: 
    'signed_out' if 
        (('Unauthenticated' in x) | 
    ('Unauthenticated' in x)) else 'signed_in')

# df.groupby('person_id', as_index=False).transform(lambda x: '' if ('anonymous' in x['peson_id']) else ','.join(x['new_ga_client_id']))

# Group by with dist

In [ ]:
sql_query = """
SELECT * 
FROM ETEXT.CLICKSTREAM.ETEXT_TELEMETRY_HIT 
LIMIT 1000;
"""

# execute and store query
cs.execute(sql_query)
df1 = cs.fetch_pandas_all()

df1.columns = [x.lower() for x in df1.columns]
df1.head()

In [ ]:
sql_query = """
SELECT * 
FROM PDI_DATA_PRODUCTS.DATA_QUALITY_INTELLIGENCE.ETEXT_USER_STATUS 
LIMIT 1000;
"""

# execute and store query
cs.execute(sql_query)
df2 = cs.fetch_pandas_all()

df2.columns = [x.lower() for x in df2.columns]
df2.head()

In [ ]:
######################## left outer join ################################################
df = df1.merge(
    df2[[
        'person_id',
        'person_status_code'
    ]],
    left_on='person_id',
    right_on='person_id',
    how='left'
)

######################## convert date columns to pandas datetime ########################
df['hit_date'] = pd.to_datetime(df['hit_date'])
df['session_start_timestamp'] = pd.to_datetime(df['session_start_timestamp'])
df['session_end_timestamp'] = pd.to_datetime(df['session_end_timestamp'])

######################## apply filters ##################################################
df[
    (
        (df.host_name == 'Study Channels') | 
        (df.event_category == 'Study Channels Full Screen Mode') | 
        (df.event_category == 'Study Channels Enrollment Flow')) |
    (
        (df.host_name == 'channels.pearson.com') | 
        (df.host_name == 'www.person.com')) |                               # Web channels domain
    (
        (df.page_name == 'Bookshelf') & 
        (df.hit_type != 'EVENT') & 
        (df.application_platform_code == 'iOS')) |                          # mobile channels domain
    (
        (df.page_name == 'Mojo Home') & 
        (df.hit_type != 'EVENT')) | 
    (
        (df.page_name == 'Mojo Home') & 
        (df.event_action == 'User Sees Container')) &
    (
        df.session_start_timestamp != df.session_end_timestamp) &           # removes bots
        (df.person_status_code == 'VALID') &                                # removes QA
        (df.hit_date >= pd.to_datetime('2022-08-01'))
]

######################## calculated columns #############################################
# get parts of date
df['week_start'] = df.hit_date.dt.week
df['month_start'] = df.hit_date.dt.month
df['quarter_start'] = df.hit_date.dt.quarter

# regex 
# +: one or more occurences of preceding element
# (): defines the scope of operators
# \: escape character
# []: character class. E.g., [abc] = (a|b|c)
df['channel_type_from_page_url'] = np.where(
    df.page_url.str.contains('https://www.pearson.com/channels/([a-z|0-9\-]+)', regex=True),
    df.page_url.str.split('/')[::-1],
    np.nan
)

df.head()

# Interactive Plot

In [1]:
import plotly.io as pio
import pandas as pd 

In [2]:
df = pd.read_csv("https://plotly.com/~public.health/17.csv")
df.head()

,Col1,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state,description,latitude,longitude,state_name
0,0,K9 Roscoe,"Phoenix Police Department, AZ","EOW: Friday, July 13, 1984",Cause of Death: Struck by vehicle,Struck by vehicle,1984-07-13,1984,True,Phoenix Police Department,AZ,Phoenix,33.448143,-112.096962,Arizona
1,1,"Police Officer Roy L. Leon, Jr.","Cotton Plant Police Department, AR","EOW: Friday, July 13, 1984",Cause of Death: Gunfire,Gunfire,1984-07-13,1984,False,Cotton Plant Police Department,AR,Little Rock,34.746613,-92.288986,Arkansas
2,2,Officer Stanley D. Pounds,"Portland Police Bureau, OR","EOW: Wednesday, July 18, 1984",Cause of Death: Automobile accident,Automobile accident,1984-07-18,1984,False,Portland Police Bureau,OR,Salem,44.938461,-123.030403,Oregon
3,3,"Enforcement Agent Ernest Joseph Gray, Jr.","Pennsylvania Public Utility Commission, PA","EOW: Friday, July 20, 1984",Cause of Death: Automobile accident,Automobile accident,1984-07-20,1984,False,Pennsylvania Public Utility Commission,PA,Harrisburg,40.264378,-76.883598,Pennsylvania
4,4,"Police Officer James W. Carozza, Jr.","Greenburgh Police Department, NY","EOW: Friday, July 20, 1984",Cause of Death: Vehicle pursuit,Vehicle pursuit,1984-07-20,1984,False,Greenburgh Police Department,NY,Albany,42.652843,-73.757874,New York


In [3]:

data = [dict(
  x = df['date'],
  autobinx = False,
  autobiny = True,
  marker = dict(color = 'rgb(68, 68, 68)'),
  name = 'date',
  type = 'histogram',
  xbins = dict(
    end = '2016-12-31 12:00',
    size = 'M1',
    start = '1983-12-31 12:00'
  )
)]


layout = dict(
  paper_bgcolor = 'rgb(240, 240, 240)',
  plot_bgcolor = 'rgb(240, 240, 240)',
  title = '<b>Shooting Incidents</b>',
  xaxis = dict(
    title = '',
    type = 'date'
  ),
  yaxis = dict(
    title = 'Shootings Incidents',
    type = 'linear'
  ),
  updatemenus = [dict(
        x = 0.1,
        y = 1.15,
        xref = 'paper',
        yref = 'paper',
        yanchor = 'top',
        active = 1,
        showactive = True,
        buttons = [
        dict(
            args = ['xbins.size', 'D1'],
            label = 'Day',
            method = 'restyle',
        ), dict(
            args = ['xbins.size', 'M1'],
            label = 'Month',
            method = 'restyle',
        ), dict(
            args = ['xbins.size', 'M3'],
            label = 'Quarter',
            method = 'restyle',
        ), dict(
            args = ['xbins.size', 'M6'],
            label = 'Half Year',
            method = 'restyle',
        ), dict(
            args = ['xbins.size', 'M12'],
            label = 'Year',
            method = 'restyle',
        )]
  )]
)

fig_dict = dict(data=data, layout=layout)

pio.show(fig_dict, validate=False)

# Observable Plot
https://observablehq.com/@d3/moving-average

# Plotly Time Series
https://plotly.com/python/time-series/

# Collapsable Network Graph
http://jsfiddle.net/ellei/zo02ve6s/